In [1]:
import numpy as np, pandas as pd
from dataclasses import dataclass

# ---- 0) Helpers ----
def ann_factor(freq='M'): return 12 if freq=='M' else 252
def annualized_ret(r): 
    n = r.shape[0]; k = ann_factor('M')
    g = (1 + r).prod()**(k/n) - 1
    return g
def annualized_vol(r): 
    return r.std() * np.sqrt(ann_factor('M'))
def sharpe(r, rf=0.0): 
    ex = r - rf/ann_factor('M')
    return ex.mean()/ex.std() * np.sqrt(ann_factor('M')) if ex.std() > 0 else np.nan
def sortino(r, rf=0.0):
    ex = r - rf/ann_factor('M')
    downside = ex[ex<0].std()
    return ex.mean()/downside * np.sqrt(ann_factor('M')) if downside and not np.isnan(downside) else np.nan
def max_drawdown(curve):
    peak = curve.cummax()
    dd = curve/peak - 1.0
    return dd.min()

def turnover(weights_df):
    # monthly turnover as 0.5 * sum |w_t - w_{t-1}| across assets
    dw = weights_df.diff().abs().sum(axis=1)
    return (0.5 * dw).mean()

# ---- 1) Vol & risk-parity ----
def rolling_vol(df_rets, win=36):
    return df_rets.rolling(win).std()

def risk_parity_weights(df_rets, win=36, eps=1e-8):
    vol = rolling_vol(df_rets, win).replace(0, np.nan)
    inv = 1.0/(vol + eps)
    w = inv.div(inv.sum(axis=1), axis=0)
    return w

# ---- 2) Macro-adaptive weights ----
def macro_weights(p, cols=('ret_eq','ret_bd','ret_gold','ret_crd'),
                  w_eq=lambda p: 1-p,
                  w_bd=lambda p: 0.60*p,
                  w_gold=lambda p: 0.40*p,
                  w_crd=lambda p: 0.30*(1-p),
                  hysteresis=None,  # e.g., {'on':0.6,'off':0.5,'k':2}
                  cash_filter=None  # e.g., {'p':0.7,'lookback':6}
                 ):
    w = pd.DataFrame(index=p.index, columns=cols, dtype=float)
    # base weights each month
    w['ret_eq']   = w_eq(p)
    w['ret_bd']   = w_bd(p)
    w['ret_gold'] = w_gold(p)
    w['ret_crd']  = w_crd(p)
    w[w<0] = 0.0
    w = w.div(w.sum(axis=1), axis=0).fillna(0.0)  # renormalize

    # optional hysteresis (binary overlay that biases towards eq or bonds/gold)
    if hysteresis:
        on, off, k = hysteresis['on'], hysteresis['off'], hysteresis['k']
        s = pd.Series(index=p.index, dtype=int)
        s.iloc[0] = 0
        run = 0
        prev_state = 0
        for t in range(1, len(p)):
            state = prev_state
            run = run+1 if ((p.iloc[t]>=on and prev_state==0) or (p.iloc[t]<=off and prev_state==1)) else 1
            if p.iloc[t]>=on and prev_state==0 and run>=k: state = 1
            if p.iloc[t]<=off and prev_state==1 and run>=k: state = 0
            s.iloc[t], prev_state = state, state
        # gently tilt: if risk-off state, shift 20% weight from eq->bd/gold
        tilt = 0.20*s
        shift = (w['ret_eq']*tilt).fillna(0)
        w['ret_eq'] -= shift
        w[['ret_bd','ret_gold']] = w[['ret_bd','ret_gold']].add(shift.values.reshape(-1,1)/2)

    w = w.div(w.sum(axis=1), axis=0).fillna(0.0)

    # optional cash filter
    if cash_filter:
        # user provides bond momentum externally
        pass  # handled in backtest via `bond_mom` input
    return w

# ---- 3) Backtest engine ----
@dataclass
class BTResult:
    returns: pd.Series
    weights: pd.DataFrame
    curve: pd.Series
    stats: dict

def backtest(returns_df, weights_df, trans_cost_bps=0.0):
    # align
    rets = returns_df.reindex(weights_df.index).dropna()
    w    = weights_df.reindex(rets.index).fillna(0.0)
    # simple monthly rebalancing at month-end
    port_ret = (w * rets).sum(axis=1)
    # turnover costs
    if trans_cost_bps>0:
        tw = w.diff().abs().sum(axis=1)*0.5
        port_ret -= tw * (trans_cost_bps/1e4)
    curve = (1+port_ret).cumprod()
    stats = {
        'AnnRet': annualized_ret(port_ret),
        'AnnVol': annualized_vol(port_ret),
        'Sharpe': sharpe(port_ret),
        'Sortino': sortino(port_ret),
        'MaxDD' : max_drawdown(curve),
        'Turnover': turnover(w)
    }
    return BTResult(port_ret, w, curve, stats)


In [5]:
import yfinance as yf
import pandas_datareader.data as web

# OOS probability series
df = pd.read_csv("../../../data/processed/recession_prob_12m_oos_multivar.csv",
                  index_col=0, parse_dates=True).rename(columns={"phat":"p_recession","y":"y"})
df = df.asfreq("ME")


# download prices and compute monthly returns
start_date = df.index[0]
tickers = ["SPY", "VFITX"]
px = yf.download(tickers, start=start_date, auto_adjust=True, progress=False)["Close"]
px = px.resample("ME").last()

ret = px.pct_change().dropna()
ret.rename(columns={'SPY': 'ret_eq', 'VFITX': 'ret_bond'}, inplace=True)

df = pd.merge(df, ret, how='inner', left_index=True, right_index=True)

tbill = web.DataReader("TB3MS", "fred", start=start_date)
tbill.rename(columns={"TB3MS": "TB3MS_rate"}, inplace=True)
tbill = tbill.resample("ME").last()

df['ret_cash'] = tbill["TB3MS_rate"] / 100 / 12
df

,p_recession,y,ret_eq,ret_bond,ret_cash
1997-02-28,0.134124,0,0.009565,0.000983,0.004175
1997-03-31,0.077201,0,-0.044123,-0.013034,0.004283
1997-04-30,0.051048,0,0.062604,0.016122,0.004300
1997-05-31,0.066194,0,0.063207,0.008343,0.004208
1997-06-30,0.068870,0,0.041102,0.010031,0.004108
...,...,...,...,...,...
2022-06-30,0.996404,0,-0.082460,-0.008270,0.001242
2022-07-31,0.996713,0,0.092088,0.019216,0.001858
2022-08-31,0.996134,0,-0.040802,-0.029086,0.002192
2022-09-30,0.995388,0,-0.092446,-0.032554,0.002608


In [10]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web

# ---- Fetch raw proxies ----
start = '1960-01-01'

# Gold
gold = web.DataReader('GOLDAMGBD228NLBM', 'fred', start)
ret_gold = gold.pct_change().fillna(0)

# ---- Merge and resample monthly ----
df = pd.concat({
    'ret_gold': ret_gold,
}, axis=1).resample('ME').last().dropna()


RemoteDataError: Unable to read URL: https://fred.stlouisfed.org/graph/fredgraph.csv?id=GOLDAMGBD228NLBM
Response Text:
b'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n    <meta charset="utf-8">\r\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n    <meta name="viewport" content="width=device-width, initial-scale=1">\r\n    <title>Error - St. Louis Fed</title>\r\n    <meta name="description" content="">\r\n    <meta name="keywords" content="">    \r\n    <link rel="stylesheet" type="text/css" href="/assets/bootstrap/dist/css/bootstrap.min.css">\r\n    <link rel="stylesheet" type="text/css" href="/css/home.min.css?1553087253">\r\n    <link rel="stylesheet" type="text/css" href="/assets/fontawesome-free/css/all.min.css">\r\n    <link rel="stylesheet" type="text/css" href="/assets/select2/dist/css/select2.min.css">\r\n    <style>p {\r\n        margin-bottom: 1.5em;\r\n    }</style>\r\n</head>\r\n<body>\r\n<link rel="preconnect" href="https://fonts.googleapis.com">\n<link r

In [ ]:
# ---- 4) Run: macro, 60/40, risk-parity ----
# Expect df with columns: ret_eq, ret_bd, ret_gold, ret_crd, p_rec
# df = <your prepared monthly dataframe>
assets = ['ret_eq','ret_bd','ret_gold','ret_crd']

# Macro weights (baseline)
W_macro = macro_weights(df['p_rec'])

# 60/40
W_6040 = pd.DataFrame(index=df.index, columns=assets, data=0.0)
W_6040['ret_eq'] = 0.60
W_6040['ret_bd'] = 0.40

# Risk parity (36m)
W_rp = risk_parity_weights(df[assets], win=36).fillna(method='bfill').fillna(0.0)

# Backtests
bt_macro = backtest(df[assets], W_macro, trans_cost_bps=0)
bt_6040  = backtest(df[assets], W_6040, trans_cost_bps=0)
bt_rp    = backtest(df[assets], W_rp,   trans_cost_bps=0)

# Metrics table
metrics = pd.DataFrame({
    'Dynamic': bt_macro.stats,
    '60/40' : bt_6040.stats,
    'RiskParity': bt_rp.stats
}).T

# --- Plot & save ---
import matplotlib.pyplot as plt
plt.figure(figsize=(9,5))
(pd.concat({
    'Dynamic': bt_macro.curve/ bt_macro.curve.iloc[0],
    '60/40':   bt_6040.curve/  bt_6040.curve.iloc[0],
    'RiskParity': bt_rp.curve/ bt_rp.curve.iloc[0],
}, axis=1)).plot(ax=plt.gca())
plt.title('Dynamic Risk-On/Off vs Benchmarks')
plt.ylabel('Growth of $1')
plt.xlabel('')
plt.tight_layout()
plt.savefig('10_dynamic_risk_on_off.png', dpi=180)

print(metrics.round(3))